In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
#QPU connection
QLMASS = True
if QLMASS:
    try:
        from qat.qlmaas import QLMaaSConnection
        connection = QLMaaSConnection()
        LinAlg = connection.get_qpu("qat.qpus:LinAlg")
        lineal_qpu = LinAlg()
    except (ImportError, OSError) as e:
        print('Problem: usin PyLinalg')
        from qat.qpus import PyLinalg
        lineal_qpu = PyLinalg()
else:
    print('User Forces: PyLinalg')
    from qat.qpus import PyLinalg
    lineal_qpu = PyLinalg()    

In [ ]:
%load_ext qat.core.magic

## 1. Quantum Multiplexors

Implementation of loading data routines using the *Lov Grover and Terry Rudolph* routines directly, using controlled rotations by state, is highly inneficient. In general the use of controlled rotations generate highly deep quantum circuits prone to errors. 
A more efficient approach is the use of Quantum Multiplexors where the only controlled gates are c-Not ones. 

The operation we want to implement is a controlled rotation on an angle $\theta_i$ controlled by a state $|i\rangle$. 

For a 2 qbits state the operator should be:

$$
\hat{U}|q_0q_1\rangle= \{ \begin{array}{ll}
      |0\rangle\hat{R}_y(\theta_0)|q_1\rangle  & if \;|q_0\rangle = |0\rangle \\
      |1\rangle\hat{R}_y(\theta_1)|q_1\rangle  & if \;|q_0\rangle = |1\rangle \\
\end{array}
$$

With a quantum multiplexor this operation can be implemented as:

$$\hat{U}(\theta_0, \theta_1)|q_0q_1\rangle= \left( \mathbf{I} \otimes \hat{R}_{y} (\frac{\theta_0+\theta_1}{2}) \right) \hat{C}_{NOT} \left(\mathbf{I} \otimes \hat{R}_{y} (\frac{\theta_0-\theta_1}{2}) \right) \hat{C}_{NOT}|q_0q_1\rangle$$

The circuit representation for this operator is:

![alt text](./QM_01.png)

As can be seen the quantum multiplexor needs only $C_{NOT}$ gates that are implemented in easier way than controlled rotations


For controlled state rotations of more angles quantum multiplexors can be implemented in a recursively way. For example if we want apply a controlled 4 angle rotation $[\theta_0, \theta_1, \theta_2, \theta_3]$ we can use following circuit:

![alt text](./QM_02.png)


In general a quantum multiplexor with $m$ control qubits can be decomposed as 2 multiplexors with $m − 1$ control qubits and 2 $\hat{C}_{NOT}$ gates:
![alt text](./QM_03.png)

In the **QuantumMultiplexors_Module** there are an implementation for a quantum multiplexor controlled by $m$ qbits and the functions neede to use them in order to load probability and a Integral function

## 2. Loading Data Gates: $\mathcal{P}$ and $\mathcal{R}$

First thing we need are the $\mathcal{P}$ and $\mathcal{R}$ for loading probability distribution and integral function respectively into a Quantum State.

In [ ]:
from AuxiliarFunctions import postprocess_results, run_job, get_histogram

In [ ]:
def p(x):
    return x*x
def f(x):
    return np.sin(x)

In [ ]:
#number of Qbits for the circuit
n_qbits = 8
#The number of bins 
m_bins = 2**n_qbits
LowerLimit = 0.0
UpperLimit = 1.0 

X, p_X = get_histogram(p, LowerLimit, UpperLimit, m_bins)
f_X = f(X)

### 2.1 Loading Probability gate: $\mathcal{P}$

The $\mathcal{P}$ is implemented by the function **load_p_gate** inside the **QuantumMultiplexors_Module**. The input should be a numpy array with the Probability distribution to load into the quantum state. The output of the function is a **qlm** *AbstractGate* customized for the input numpy array which load it into the quantum state. 

First we need to configurate the $\mathcal{P}$ by providing the discretized probability to load

In [ ]:
from qat.lang.AQASM import Program, H, QRoutine
from QuantumMultiplexors_Module import load_p_gate

In [ ]:
p_gate = load_p_gate(p_X)
%qatdisplay p_gate --depth 1

Now we can create a function that creates a quantum program by giving to it an *AbstractGate* or a *QRoutine*.

In [ ]:
def create_qprogram(quantum_gate):
    """
    Creates a Quantum Program from an input qlm gate or routine

    Parameters
    ----------

    quantum_gate : QLM gate or QLM routine

    Returns
    ----------
    q_prog: QLM Program.
        Quantum Program from input QLM gate or routine
    """
    q_prog = Program()
    qbits = q_prog.qalloc(quantum_gate.arity)
    q_prog.apply(quantum_gate, qbits)
    return q_prog


In [ ]:
Qprog = create_qprogram(p_gate)
circuit_P = Qprog.to_circ(submatrices_only=True)

In [ ]:
%qatdisplay circuit_P --depth 0

Following cell simulates the circuit an postprocces result in order to test in probability was properly loaded. The results of the simulation is given to the **PostProcessResults** which create a pandas dataframe with the posible states and teh correspondings probabilities and amplitudes.

In [ ]:
jobP = circuit_P.to_job()
resultP = lineal_qpu.submit(jobP)
P_results = postprocess_results(run_job(resultP))

In [ ]:
P_results.head()

In order to check if the operation was done properly we can compare the outpus probability of each state and the probabilities wanted to load:

In [ ]:
np.isclose(P_results['Probability'], p_X).all()

### 2.2 Loading Integral Function gate: $\mathcal{R}$


The $\mathcal{R}$ is implemented by an Abstract qlm gate called **load_r_gate**  inside the **QuantumMultiplexors_Module**.. The input should be a numpy array with the function which integral want to be load into the quantum state. The output of the function is a **qlm** *AbstractGate* customized for the input numpy array which load the integral into the last qbit of the quantum state. 


The length of this array should be: $m=2^n$ where n is an integer. The created gate is a $n+1$ gate where the last qbit codifies the integral of the function.

Following **load_integral_routine** function creates a complete Quantum routine for loading Integral of discretized function $f(x)$ using the programed $\mathcal{R}$ gate. The size of the function array should be $2^n$ and the circuit will have $n+1$ qbit where the last one qbit will enconde the integral of the $f(x)$. This function is neede becaseu we need to generate an initial state of equiprobably combiantion of states

In [ ]:
def load_integral_routine(r_gate):
    """
    Creates a Quantum Program for loading the integral of an input
    function given as a numpy array using Quantum Multiplexors (QM).

    Parameters
    ----------

    r_gate : QLM AbstractGate
        Customized AbstractGatel for loading integral of a function f(x)

    Returns
    ----------
    q_prog: QLM Program
        Quantum Program for loading integral of the input function
    r_gate: QLM AbstractGate
        Customized AbstractGate for loading integral using QM
    """
    q_rout = QRoutine()
    qbits = q_rout.new_wires(r_gate.arity)
    #equiprobable superposition of states
    for i in range(len(qbits)-1):
        q_rout.apply(H, qbits[i])
    q_rout.apply(r_gate, qbits)
    return q_rout


First we need to configurate the $\mathcal{R}$ operator with the discretized function

In [ ]:
from QuantumMultiplexors_Module import load_r_gate

In [ ]:
r_gate = load_r_gate(f_X)
%qatdisplay r_gate --depth 0

Now we can provide this *load_integral_routine* to the **create_qprogram**

In [ ]:
Qprog = create_qprogram(load_integral_routine(r_gate))
circuit_R = Qprog.to_circ(submatrices_only=True)

In [ ]:
%qatdisplay circuit_R --depth 0

In this case we are only interested in the last qbit of the circuit that encodes the desIred integral. Following cell simulates the circuit from *load_integral_program* and post process the results using *PostProcessResults* function which ouputs a pandas dataframe with the posible states (measurement of the last qbit) and the corresponding probability

In [ ]:
jobR = circuit_R.to_job(qubits = [n_qbits])
resultR = lineal_qpu.submit(jobR)
R_results= postprocess_results(run_job(resultR))

In [ ]:
#Integral of f(x)
MeasurementIntegral = R_results['Probability'][1]*2**(n_qbits)
np.isclose(MeasurementIntegral, sum(f_X))

### 2.3 Load Complete Data

Now we are going to use $\mathcal{P}$ and $\mathcal{R}$ implemented gates in order to load $E_{x\sim p}(f)$ in a quantum circuit. 

$$E_{x\sim p}(f) = \sum_{x=0}^{2^n-1} p(x)f(x)dx$$

For this the **load_pr_gate**  inside **QuantumMultiplexors_Module** was created. Providing the 2 operator gates $\mathcal{P}$ and $\mathcal{R}$ the function creates a third *Abstract Gate* with the two operators apply consecutively

In [ ]:
from QuantumMultiplexors_Module import load_pr_gate

In [ ]:
pr_gate = load_pr_gate(p_gate, r_gate)
%qatdisplay pr_gate --depth 1

Now we can provide this *load_integral_routine* to the **create_qprogram**

In [ ]:
Qprog = create_qprogram(pr_gate)
circuitPR = Qprog.to_circ(submatrices_only=True)

In [ ]:
%qatdisplay circuitPR --depth 0

In [ ]:
jobPR = circuitPR.to_job(qubits = [n_qbits])
resultPR = lineal_qpu.submit(jobPR)
PR_results = postprocess_results(run_job(resultPR))

As explained before the $E_{x\sim p}(f)$ will be loaded in the $|1\rangle$ state of the last qbit of the circuit

In [ ]:
#Integral of f(x)
MeasurementIntegral = PR_results['Probability'][1]
print('MeasurementIntegral: {}'.format(MeasurementIntegral))
print('Integral p(x)f(x): {}'.format(sum(p_X*f_X)))

In [ ]:
np.isclose(MeasurementIntegral, sum(p_X*f_X))

## 3 Sumary and Important notes

In **01_Dataloading_Module_Use** Notebook the *dataloading_module* and *dataloading_module_examples* were explained and used. 
Present Notebook try to mimic the **01_Dataloading_Module_Use** one but using and explained **QuantumMultiplexors_Module** and **QuantumMultiplexors_Module_examples**. 

The functions and gates implemented in *dataloading_module* and in the **QuantumMultiplexors_Module** have the same functionality: **load data, probability functions and integral functions** in a quantum state but the implementation was changed:
* In the *dataloading_module*: several controlled by states rotations were straightoforward implemented in order to create the gates to load the data.
* In the **QuantumMultiplexors_Module**: the loading data gates were implemented in a much more efficient way using **quantum multiplexors** where the controlloed rotations are subsituted by simple (and intelligent) qbit Rotations and $\hat{C}_{NOT}$ gates

## 4. File Scripts

All the functions generated in this Notebook were stored in a python file, **QuantumMultiplexors_Module_examples.py** in order to use it in an easy way:


In [ ]:
from AuxiliarFunctions import get_histogram, postprocess_results, run_job
def p(x):
    return x*x
def f(x):
    return np.sin(x)

#number of Qbits for the circuit
n_qbits = 6
#The number of bins 
m_bins = 2**n_qbits
LowerLimit = 0.0
UpperLimit = 1.0 
X, p_X = get_histogram(p, LowerLimit, UpperLimit, m_bins)
f_X = f(X)

In [ ]:
from QuantumMultiplexors_Module_examples import load_probability_program, load_integral_program, expectation_loading_data

### 4.1 Probability Load

In [ ]:
Qprog, _ = load_probability_program(p_X)
circuit_P = Qprog.to_circ(submatrices_only=True)
%qatdisplay circuit_P

In [ ]:
jobP = circuit_P.to_job()
resultP = lineal_qpu.submit(jobP)
P_results = postprocess_results(run_job(resultP))
np.isclose(P_results['Probability'], p_X).all()

In [ ]:
P_results['Probability']

In [ ]:
p_X

### 4.2 Integral Load

In [ ]:
Qprog, _ = load_integral_program(f_X)
circuit_R = Qprog.to_circ(submatrices_only=True)
%qatdisplay circuit_R

In [ ]:
jobF = circuit_R.to_job(qubits = [n_qbits])
resultF = lineal_qpu.submit(jobF)
F_results = postprocess_results(run_job(resultF))
MeasurementIntegral = F_results['Probability'][1]*2**(n_qbits)
np.isclose(MeasurementIntegral, sum(f_X))

### 4.3 Complete Load

In [ ]:
Qprog, _, _ = expectation_loading_data(p_X, f_X)
circuitPR = Qprog.to_circ(submatrices_only=True)
%qatdisplay circuitPR

In [ ]:
jobPF = circuitPR.to_job(qubits = [n_qbits])
resultPF = lineal_qpu.submit(jobPF)
PF_results = postprocess_results(run_job(resultPF))
MeasurementIntegral = PF_results['Probability'][1]
np.isclose(MeasurementIntegral, sum(p_X*f_X))